In [1]:
from dataloader import SignalDataset,dataloader 
from param import dataset_path,sample_universe_size,sampling_rate
from utils import prepare_data, mix_signals,load_audio,load_music
import torch
from torch.utils.data import ConcatDataset
import os
from param import (
    dataset_path,
    sample_universe_size,
    data_dir,
    hyper_parameters as hp,
    model_1,
)
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
#Call this code in train and evaluate functions (to be deleted later)
data_dir = r"../data"
model_dir = r"../models"
data_file = os.path.join(data_dir, "dataset.pth")
if "dataset.pth" in os.listdir(data_dir):
    combined_dataset = torch.load(data_file)
else:
    # Prepare data
    combination_paths = prepare_data(dataset_path)
    sampled_df = combination_paths.sample(
        frac=sample_universe_size, random_state=42, ignore_index=True
    )
    datasets_music = SignalDataset(sampled_df, data_type="music")
    datasets_speech = SignalDataset(sampled_df, data_type="speech")
    datasets_mixture = SignalDataset(sampled_df, data_type="mixture")
    combined_dataset = ConcatDataset(
        [datasets_music, datasets_speech, datasets_mixture]
    )
    torch.save(combined_dataset, data_file)


In [3]:
train_loader, test_loader = dataloader(
    datasets=combined_dataset,
    train_ratio=hp["train_ratio"],
    train_batch_size=hp["batch_size"],
    test_batch_size=1,
)

In [5]:
# Assuming your model architecture is defined in a separate file or module
from model_architecture import MtlCascadeModel
from param import hyper_parameters as hp
# Path to the state dict file
state_dict_path = '/home/hparashar/mtech_projects/speech_understanding/Speech-Understanding-Minor/model/model_0.01.pth'
hyper_parameters = {
    "n_layers": 1,
    "sp_hidden_nodes": 20,
    "n_sp_hidden_lyrs": 1,
    "mu_hidden_nodes": 20,
    "n_mu_hidden_lyrs": 1,
    "smr_hidden_nodes": 20,
    "n_smr_hidden_lyrs": 1,
    "n_epochs": 100,
    "batch_size": 20,
    "train_ratio": 0.8,
}
# Create an instance of your model
model = MtlCascadeModel(hyper_parameters)

# Load the state dict onto the model
state_dict = torch.load(state_dict_path)
model.load_state_dict(state_dict)

# Now, you can use your model for inference or further training


<All keys matched successfully>

In [64]:
from param import targ_dict
def evaluate(model, test_loader):
    model.eval()
    su_predictions = []
    mu_predictions = []
    smr_predictions = []
    su_target = []
    mu_target = []
    smr_target = []
    for data in test_loader:
        feature, target = data
        su_pred,mu_pred,smr_pred = model(feature)
        su_predictions.append((su_pred  > 0.5).float())
        mu_predictions.append((mu_pred  > 0.5).float())
        smr_predictions.append((smr_pred  > 0.5).float())
        su_target.append(float(targ_dict[target[0]][0]))
        mu_target.append(float(targ_dict[target[0]][1]))
        smr_target.append(float(targ_dict[target[0]][2]))

    return su_predictions, mu_predictions, smr_predictions, su_target, mu_target, smr_target

In [65]:
su_predictions, mu_predictions, smr_predictions, su_target, mu_target, smr_target =evaluate(model, test_loader)


In [70]:
def calculate_metrics(predictions, targets):
    # Convert predictions and targets to tensors if they're not already
    if not isinstance(predictions, torch.Tensor):
        predictions = torch.tensor(predictions)
    if not isinstance(targets, torch.Tensor):
        targets = torch.tensor(targets)

    # Calculate true positives, false positives, and false negatives
    true_positives = torch.sum(predictions * targets).float()
    false_positives = torch.sum(predictions * (1 - targets)).float()
    false_negatives = torch.sum((1 - predictions) * targets).float()

    # Calculate precision
    precision = true_positives / (true_positives + false_positives + 1e-10)

    # Calculate recall
    recall = true_positives / (true_positives + false_negatives + 1e-10)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)
    accuracy = torch.sum(predictions == targets).float() / targets.numel()
    return precision.item(), recall.item(), f1_score.item(),accuracy.item()




In [71]:
for item in ["speech","music","mixed"]:
    su_predictions, mu_predictions, smr_predictions, su_target, mu_target, smr_target
    if item == "speech":
        predictions = su_predictions
        targets = su_target
    elif item == "music":
        predictions = mu_predictions
        targets = mu_target
    else:
        predictions = smr_predictions
        targets = smr_target
    precision, recall, f1_score,accuracy  = calculate_metrics(predictions, targets)
    print(f"Precision for {item}:", precision)
    print(f"Recall for {item}:", recall)
    print(f"F1 score for {item}:", f1_score)
    print(f"Accuracy for {item}:", accuracy)
    print("-------------------------------------------------\n")

Precision for speech: 0.9464285969734192
Recall for speech: 0.9137930870056152
F1 score for speech: 0.9298245906829834
Accuracy for speech: 0.9526627063751221
-------------------------------------------------

Precision for music: 0.698113203048706
Recall for music: 0.7708333134651184
F1 score for music: 0.7326732873916626
Accuracy for music: 0.8402366638183594
-------------------------------------------------

Precision for mixed: 0.8684210777282715
Recall for mixed: 0.523809552192688
F1 score for mixed: 0.6534653306007385
Accuracy for mixed: 0.7928994297981262
-------------------------------------------------



In [ ]:
def calculate_metrics(predictions, targets):
    # Convert predictions and targets to tensors if they're not already
    if not isinstance(predictions, torch.Tensor):
        predictions = torch.tensor(predictions)
    if not isinstance(targets, torch.Tensor):
        targets = torch.tensor(targets)

    # Calculate true positives, false positives, and false negatives
    true_positives = torch.sum(predictions * targets).float()
    false_positives = torch.sum(predictions * (1 - targets)).float()
    false_negatives = torch.sum((1 - predictions) * targets).float()

    # Calculate precision
    precision = true_positives / (true_positives + false_positives + 1e-10)

    # Calculate recall
    recall = true_positives / (true_positives + false_negatives + 1e-10)

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-10)

    # Calculate accuracy
    

    return precision.item(), recall.item(), f1_score.item(), accuracy.item()